In [ ]:
import numpy as np
import time
# import matplotlib.pyplot as plt
# import matplotlib.cm as cmx
# import matplotlib.colors as colors
# from scipy.optimize import linprog
# from gurobipy import *
import pickle
## I import sys to kill the program if an option is not correct.
import sys
import os
import csv

from RhoAndBeta import CalcRhoAndBetaVectors
from UtilitiesOptimization import SubgrAlgSavPrimDualObjInd, \
    SubgrAlgSavPrimDualObjFn_L2Ind

In [ ]:
## Read the data created in CreateDataJupNot
suffix='InstanceInfo/Ins1LongRun/'
num_impressions=pickle.load(open(suffix+'num_impressions'+'.p',"rb"))
numCampaigns=pickle.load(open(suffix+'numCampaigns'+'.p',"rb"))
num_edges=pickle.load(open(suffix+'num_edges'+'.p',"rb"))
index_Imps=pickle.load(open(suffix+'index_Imps'+'.p',"rb"))
index_sizeCamps=pickle.load(open(suffix+'index_sizeCamps'+'.p',"rb"))
index_startCamp=pickle.load(open(suffix+'index_startCamp'+'.p',"rb"))
vector_m=pickle.load(open(suffix+'vector_m'+'.p',"rb"))
vector_maxbid=pickle.load(open(suffix+'vector_maxbid'+'.p',"rb"))
vector_r=pickle.load(open(suffix+'vector_r'+'.p',"rb"))
vector_s=pickle.load(open(suffix+'vector_s'+'.p',"rb"))
ext_s=pickle.load(open(suffix+'ext_s'+'.p',"rb"))
adverPerImp=pickle.load(open(suffix+'adverPerImp'+'.p',"rb"))
UB_bidsPerImp = pickle.load(open(suffix+'UB_bids'+'.p',"rb"))
vector_ctr=pickle.load(open(suffix+'vector_ctr'+'.p',"rb"))
vector_rctr=pickle.load(open(suffix+'vector_rctr'+'.p',"rb"))
probImp=pickle.load(open(suffix+'probImp'+'.p',"rb"))

In [ ]:
## If this parameter is true first price auctions (\beta_i(b) =b) are used, otherwise second price.
firstPrice = True
## If this parameter is true a parameterSearch will be performd and .csv will be made,
parameterSearch = False
## We use $\tau_k = 1/m_k$
expForTau=-1.0
tau = np.power(vector_m,expForTau)
UB_bids = UB_bidsPerImp[index_Imps]

suffix='ResultsDG/'
current_directory = os.getcwd()
results_directory = os.path.join(current_directory, suffix)
if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# Parameter Search

This parameter search looks for a constant $C$, such that the dual method shows 
empirical convergence. Given that here we only look for training convergence, we don't 
need to cross-validate or similar.

In [ ]:
if parameterSearch:
    input_var = input("This will execute a simple parameter search.\
    If you just wanted to run just a full long run do parameterSearch= False  \
    and kill this execution. To continue press enter.")
    c=np.array([0.5,0.1,0.05,0.01,0.005,0.001,0.0005,0.0001,0.00005,0.00001])
    p_grad_Type = 0
    num_it = 1000
    it_per_cal = 250
    init_lam=np.zeros((numCampaigns))
    alphas_pure=np.fromfunction(lambda i, j: (1/(np.sqrt(i + 1))), \
        (num_it, 1), dtype=int)[:,0] 
    
#     dualObjFn, primalObjFn, dualObjFnAvg, primalObjFnAvg, budget_used, \
#     budget_LamAvgUse, dual_vars, dual_varsAvg = [], [], [], [], [], [], [], []

    nameResults='ParameterSearch'+'It_'+str(num_it)
    f = open(suffix+nameResults+'.csv', 'wt')
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow( ('Auction Type', 'Problem Type', 'Ite', 'Cte', 'DualFnValue',\
        'PrimalValue','DualFnValueAvg','PrimalFnValueAvg') )
    
    print('Using First Price Auctions')
    firstPrice = True
    for c_val in c:
        print('c_val: ',c_val, end =', Methods: ')
        print('Indicator Case', end =' ,')
        p_grad_Type = 0
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjInd(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, \
            vector_m, num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,\
            UB_bids, firstPrice, adverPerImp, it_per_cal, p_grad_Type)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('First price', 'Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 penalization wout indicator', end =' ,')
        p_grad_Type = 1
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, False)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('First price', 'L2 Wout Ind', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 with indicator')
        p_grad_Type = 2
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, True)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('First price', 'L2 + Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))

            
    print('Using Second Price Auctions')
    firstPrice = False
    for c_val in c:
        print('c_val: ',c_val, end =', Methods: ')
        print('Indicator Case', end =' ,')
        p_grad_Type = 0
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjInd(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, \
            vector_m, num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,\
            UB_bids, firstPrice, adverPerImp, it_per_cal, p_grad_Type)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('Second price', 'Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 penalization wout indicator', end =' ,')
        p_grad_Type = 1
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, False)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('Second price', 'L2 Wout Ind', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 with indicator')
        p_grad_Type = 2
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, True)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('Second price', 'L2 + Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
    f.close()

# The following value are obtained just by Looking at The Parameter Search .csv

In [ ]:
## Best constants
consBestFP = [0.0001, 0.5, 0.5]
consBestSP = [0.0001, 0.5, 0.5]

# Let's run the methods for a long time

Remember that the methods only starts saving the values for the average dual values when half of the iteration has be run. 

In [ ]:
## Number of Iterations to Run
num_it = 10000
## Number of Interations to Save the Primal and Dual Values
it_per_cal = 100
## RunLongExperiment
runLongExp = True

In [ ]:
if runLongExp:
    input_var = input("This will run the Experiment with the constants as in 'consBestFP' and 'consBestSP'.\
    To continue press enter.")
    init_lam=np.zeros((numCampaigns))
    alphas_pure=np.fromfunction(lambda i, j: (1/(np.sqrt(i + 1))), \
        (num_it, 1), dtype=int)[:,0] 

    nameResults='LongRun'+'It_'+str(num_it)
    f = open(suffix+nameResults+'.csv', 'wt')
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow( ('Auction Type', 'Problem Type', 'Ite', 'Cte', 'DualFnValue',\
        'PrimalValue','DualFnValueAvg','PrimalFnValueAvg') )
    
    print('Running '+str(num_it)+' saving every '+str(it_per_cal)+' iterations.')
    
    ## Matrices with Primal dual values for different p(\cdot) and auction type combinations
    matFP_Ind, matFP_L2, matFP_L2Ind, matSP_Ind, matSP_L2, matSP_L2Ind =\
    np.zeros((int(num_it/it_per_cal),4)), np.zeros((int(num_it/it_per_cal),4)),\
    np.zeros((int(num_it/it_per_cal),4)), np.zeros((int(num_it/it_per_cal),4)),\
    np.zeros((int(num_it/it_per_cal),4)), np.zeros((int(num_it/it_per_cal),4))
    
    print('Using First Price Auctions')
    firstPrice = True
    
    print('Methods: Indicator', end = ', ')
    p_grad_Type = 0
    alphas=consBestFP[0]*alphas_pure
    [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
        primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjInd(\
        init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, \
        vector_m, num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,\
        UB_bids, firstPrice, adverPerImp, it_per_cal, p_grad_Type)
    for t in range(len(budget_used)):
        matFP_Ind[t,:] = [dual_FnValues[t],primal_GivenMu[t],dual_AvgLamFnValues[t],\
            primal_AvgLamGivenMu[t]]
        writer.writerow(('First price', 'Indicator', (t+1)*it_per_cal,consBestFP[0],dual_FnValues[t],\
            primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
    print('L2 penalization wout indicator', end = ', ')
    p_grad_Type = 1
    alphas=consBestFP[1]*alphas_pure
    [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
        primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg] = SubgrAlgSavPrimDualObjFn_L2Ind(\
        init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
        num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps, UB_bids, firstPrice,\
        adverPerImp, it_per_cal, p_grad_Type, tau, False)
    for t in range(len(budget_used)):
        matFP_L2[t,:] = [dual_FnValues[t],primal_GivenMu[t],dual_AvgLamFnValues[t],\
            primal_AvgLamGivenMu[t]]
        writer.writerow(('First price', 'L2 Wout Ind', (t+1)*it_per_cal,consBestFP[1],dual_FnValues[t],\
            primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
    print('L2 with indicator')
    p_grad_Type = 2
    alphas = consBestFP[2]*alphas_pure
    [dual_FnValues, primal_GivenMu, budget_used, dual_vars, dual_AvgLamFnValues, primal_AvgLamGivenMu,\
        budget_LamAvgUse, dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(init_lam, num_it, alphas,\
        vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m, num_impressions, numCampaigns,\
        num_edges, index_sizeCamps, index_Imps, UB_bids, firstPrice, adverPerImp, it_per_cal, p_grad_Type,\
        tau, True)
    for t in range(len(budget_used)):
        matFP_L2Ind[t,:] = [dual_FnValues[t],primal_GivenMu[t],dual_AvgLamFnValues[t],\
            primal_AvgLamGivenMu[t]]
        writer.writerow(('First price', 'L2 + Indicator', (t+1)*it_per_cal,consBestFP[2],dual_FnValues[t],\
            primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))

    print('Using Second Price Auctions')
    firstPrice = False
    
    print('Methods: Indicator Case', end =' ,')
    p_grad_Type = 0
    alphas=consBestSP[0]*alphas_pure
    [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjInd(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, \
            vector_m, num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,\
            UB_bids, firstPrice, adverPerImp, it_per_cal, p_grad_Type)
    for t in range(len(budget_used)):
        matSP_Ind[t,:] = [dual_FnValues[t],primal_GivenMu[t],dual_AvgLamFnValues[t],\
            primal_AvgLamGivenMu[t]]
        writer.writerow(('Second price', 'Indicator', (t+1)*it_per_cal,consBestSP[0],dual_FnValues[t],\
            primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
    print('L2 penalization wout indicator', end =' ,')
    p_grad_Type = 1
    alphas=consBestSP[1]*alphas_pure
    [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps, UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, False)
    for t in range(len(budget_used)):
        matSP_L2[t,:] = [dual_FnValues[t],primal_GivenMu[t],dual_AvgLamFnValues[t],\
            primal_AvgLamGivenMu[t]]
        writer.writerow(('Second price', 'L2 Wout Ind', (t+1)*it_per_cal,consBestSP[1],dual_FnValues[t],\
            primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
    print('L2 with indicator')
    p_grad_Type = 2
    alphas=consBestSP[2]*alphas_pure
    [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
        primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
        init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
        num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps, UB_bids, firstPrice,\
        adverPerImp, it_per_cal, p_grad_Type, tau, True)
    for t in range(len(budget_used)):
        matSP_L2Ind[t,:] = [dual_FnValues[t],primal_GivenMu[t],dual_AvgLamFnValues[t],\
            primal_AvgLamGivenMu[t]]
        writer.writerow(('Second price', 'L2 + Indicator', (t+1)*it_per_cal,consBestSP[2],dual_FnValues[t],\
            primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
        
    ## Let's save the matrices as pickle objects to
    pickle.dump(matFP_Ind,open(suffix+'matFP_Ind'+'.p',"wb"))
    pickle.dump(matFP_L2,open(suffix+'matFP_L2'+'.p',"wb"))
    pickle.dump(matFP_L2Ind,open(suffix+'matFP_L2Ind'+'.p',"wb"))
    pickle.dump(matSP_Ind,open(suffix+'matSP_Ind'+'.p',"wb"))
    pickle.dump(matSP_L2,open(suffix+'matSP_L2'+'.p',"wb"))
    pickle.dump(matSP_L2Ind,open(suffix+'matSP_L2Ind'+'.p',"wb"))
    ## Close the writer
    f.close()

In [ ]:
if runLongExp:
    pickle.dump(matFP_Ind,open(suffix+'matFP_Ind'+'.p',"wb"))
    pickle.dump(matFP_L2,open(suffix+'matFP_L2'+'.p',"wb"))
    pickle.dump(matFP_L2Ind,open(suffix+'matFP_L2Ind'+'.p',"wb"))
    pickle.dump(matSP_Ind,open(suffix+'matSP_Ind'+'.p',"wb"))
    pickle.dump(matSP_L2,open(suffix+'matSP_L2'+'.p',"wb"))
    pickle.dump(matSP_L2Ind,open(suffix+'matSP_L2Ind'+'.p',"wb"))

In [ ]:
matFP_Ind = pickle.load(open(suffix+'matFP_Ind'+'.p', "rb"))
matFP_L2 = pickle.load(open(suffix+'matFP_L2'+'.p', "rb"))
matFP_L2Ind = pickle.load(open(suffix+'matFP_L2Ind'+'.p', "rb"))
matSP_Ind = pickle.load(open(suffix+'matSP_Ind'+'.p', "rb"))
matSP_L2 = pickle.load(open(suffix+'matSP_L2'+'.p', "rb"))
matSP_L2Ind = pickle.load(open(suffix+'matSP_L2Ind'+'.p', "rb"))